In [4]:
import scipy as sp
import numpy as np

In [ ]:
### Winner's curse correction
WC_correction <- function(BETA,        # Effect size 
                          SE,          # Standard Error
                          alpha=5e-8){ # Significance threshold
  
  Q     <- qchisq(alpha,df=1,lower.tail=FALSE)
  cs    <- sqrt(Q)
  
  bias  <- function(betaTrue,betaObs,se){
    z   <- betaTrue/se
    num <- dnorm(z-cs)-dnorm(-z-cs)
    den <- pnorm(z-cs)-pnorm(-z-cs)
    return(betaObs-betaTrue + se * num/den)
  }
  
  solveBetaTrue <- function(betaObs,se){
    md <- uniroot(f=function(b) bias(b,betaObs,se),lower=-100,upper=100)
    return(md$root)
  }
  
  BETA_corrected  <- sapply(1:length(BETA),function(i) solveBetaTrue(BETA[i],SE[i]))
  return(BETA_corrected)
}

In [28]:
sp.stats.chi2.ppf(1-5e-8,df=1)

29.716785488631203

In [63]:
def WC_correct(beta, se, sig_level=5e-8):
    
    c2 = sp.stats.chi2.ppf(1-sig_level,df=1)
    c = np.sqrt(c2)
    # the equation to solve
    
    def bias(beta_T,beta_O,se):
        z = beta_T / se
        numerator = sp.stats.norm.pdf(z - c) - sp.stats.norm.pdf(- z - c)
        denominator = sp.stats.norm.cdf(z - c) + sp.stats.norm.cdf(- z - c)
        return beta_T + se * numerator / denominator - beta_O
    minimum = sp.optimize.brentq(lambda x : bias(x,beta,se),a=-100,b=100, maxiter=1000)
    
    return minimum

In [64]:
WC_correct(0.05510736, 0.008186396)

0.05271738791297707

In [61]:
WC_correct(0.15,0.02)

5.451310437741663
solving


0.1488781270128764

In [45]:
sp.stats.chi2.ppf(1-5e-8,df=1)

29.716785488631203